In [1]:
class LambdaError(Exception):
    __errmsg = [
        "unrecognised error",
    ]

    def __init__(self, errDescription):
        if isinstance(errDescription, int):
            try:
                self._msg = LambdaError.__errmsg[errDescription]
            except:
                self._msg = LambdaError.__errmsg[0]
        elif isinstance(errDescription, str):
            self._msg = errDescription
        else:
            self._msg = LambdaError.__errmsg[0]
        super().__init__(self._msg)


class Var:
    __cvar = 0

    def __init__(self):
        self._data = Var.__cvar
        Var.__cvar += 1

    def __str__(self):
        return f"#{self._data}"

    def __eq__(self, another):
        if isinstance(another, Var):
            return self._data == another._data
        raise LambdaError("Var.__eq__ waits for an instance of Var"
                          f", but it received '{another}'")


class Term:  # the basic abstract class for representing a term

    @property
    def kind(self):  # returns the kind of the term
        if isinstance(self, Atom):
            return "atom"
        if isinstance(self, Application):
            return "application"
        if isinstance(self, Abstraction):
            return "abstraction"

    def __str__(self):
        if self.kind == "atom":
            return f"{self._data}"
        if self.kind == "application":
            return f"({self._data[0]} {self._data[1]})"
        else:  # self.kind == "absraction"
            return f"(λ{self._data[0]}. {self._data[1]})"

    # def __eq__(self, another):
    #     if isinstance(another, Term):
    #         if self.kind != another.kind:
    #             return False
    #         return self._data == another._data
    #     else:
    #         raise LambdaError(3)
    def __eq__(self, other):
        if not isinstance(other, Term):
            return False
        if self.kind == "atom" and other.kind == "atom":
            return self._data == other._data
        if self.kind == "application" and other.kind == "application":
            return self._data[0] == other._data[0] and self._data[1] == other._data[1]
        if self.kind == "abstraction" and other.kind == "abstraction":
            return self._data[0] == other._data[0] and self._data[1] == other._data[1]
        return False

    def call_as_method(self, fun, *args):
        return fun(self, *args)

    @property
    def is_beta_redex(self):
        """:return: bool is the term is a beta-redex"""
        return (self.kind == "application") and (self._data[0].kind == "abstraction")

    @property
    def redexes(self):
        """:return: list of all beta-redexes in the term"""
        if self.kind == "atom":
            return []
        if self.kind == "abstraction":
            return self._data[1].redexes
        # self is App:
        redexes_list = [self] if self.is_beta_redex else []
        redexes_list += self._data[0].redexes + self._data[1].redexes
        return redexes_list

    @property
    def _vars(self):
        """
        Here, keys of the external dictionary are the variables that
        are occurred in 'self', and values of the internal dictionaries
        relate respectively to the numbers of free and bound occurrences
        of the variables.
        :return: dict[Var, dict[('free'/'bound'), int]]
        """
        if self.kind == "atom":
            return {self._data: {"free": 1, "bound": 0}}
        if self.kind == "application":
            vars_, auxvars_ = dict(self._data[0]._vars), self._data[1]._vars
            for var_ in auxvars_:
                try:
                    for key_ in ("free", "bound"):
                        vars_[var_][key_] += self._data[1]._vars[var_][key_]
                except KeyError:
                    vars_[var_] = dict(self._data[1]._vars[var_])
            return vars_
        # self is Abstraction:
        vars_ = dict(self._data[1]._vars)
        try:
            vars_[self._data[0]]["bound"] += vars_[self._data[0]]["free"]
            vars_[self._data[0]]["free"] = 0
        except KeyError:
            pass
        return vars_

    @property
    def vertices_number(self):
        """:return: the number of nodes in the tree representation the lambda term"""
        if self.kind == "atom":
            return 1
        if self.kind == "application":
            return 1 + self._data[0].vertices_number + self._data[1].vertices_number
        # self is Abstraction
        return 1 + self._data[1].vertices_number

    def normalize(self, strategy):
        """
        :param strategy: OneStepStrategy
        :return: tuple of the normal form of the term and number of steps of betta reduction
        """
        term = self._update_bound_vars()
        count_steps = 0
        while len(term.redexes) > 0:
            term = term._beta_conversion(strategy)._update_bound_vars()
            count_steps += 1
            # computation limitation
            if (term.vertices_number > 7_000) or (count_steps > 400):
                return term, float("inf")
        return term, count_steps

    def _beta_conversion(self, strategy):
        """
        :param strategy: OneStepStrategy
        :return: term with redex eliminated using the given strategy
        """
        index = strategy.redex_index(self)
        subterm_ = self.subterm(index)
        reduced_term = subterm_._remove_outer_redex()
        return self.set_subterm(index, reduced_term)

    def subterm(self, index: int):
        """
        By representing the term as a tree, a subtree is returned,
        which is also a lambda term.
        The vertex of this subtree has a given index in the topological
        sorting of the vertices of the original term.
        :param index: int subterm index
        :return: subterm: Term
        """
        if index == 1:
            return self
        if self.kind == "atom":
            raise ValueError("index value is incorrect")
        elif self.kind == "application":
            if self._data[0].vertices_number + 1 >= index:
                return self._data[0].subterm(index - 1)
            else:
                return self._data[1].subterm(index - self._data[0].vertices_number - 1)
        else:
            return self._data[1].subterm(index - 1)

    def set_subterm(self, index: int, term):
        """
        By representing the term as a tree, a subtree is set, which is also a lambda term.
        The vertex of this subtree has a given index in the topological sorting of the vertices of the original term.
        :param index: subterm index
        :param term: λ-term to which the subterm will be replaced
        :return: updated λ-term
        """
        if index == 1:
            return  term

        if self.kind == "atom":
            raise ValueError("index value is incorrect")
        elif self.kind == "application":
            if self._data[0].vertices_number + 1 >= index:
                return Application(self._data[0].set_subterm(index - 1, term), self._data[1])
            else:
                return Application(self._data[0],
                                   self._data[1].set_subterm(index - self._data[0].vertices_number - 1, term))
        else:
            return Abstraction(self._data[0], self._data[1].set_subterm(index - 1, term))

    def _update_bound_vars(self):
        """:return: λ-term with updated bound variables"""
        if self.kind == "atom":
            return self
        if self.kind == "application":
            return Application(
                self._data[0]._update_bound_vars(),
                self._data[1]._update_bound_vars()
            )
        # self is abstraction
        new_var = Var()
        return Abstraction(
            new_var,
            self._data[1]._replace_variable(self._data[0], Atom(new_var))._update_bound_vars()
        )

    def _remove_outer_redex(self):
        """Apply the betta conversion to the lambda term, removing the outer betta redex"""
        if self.is_beta_redex:
            head = self._data[0]._data[0]
            body = self._data[0]._data[1]
            return body._replace_variable(head, self._data[1])
        else:
            return self

    def _replace_variable(self, var: Var, term):
        """Return λ-term with replaced variable"""
        if self.kind == "atom":
            return term if self._data == var else self
        if self.kind == "application":
            return Application(self._data[0]._replace_variable(var, term),
                               self._data[1]._replace_variable(var, term))
        # self is abstraction
        return Abstraction(self._data[0], self._data[1]._replace_variable(var, term))



class Atom(Term):  # the class of terms created with the first rule

    def __init__(self, v):
        if isinstance(v, Var):
            self._data = v
        else:
            raise LambdaError("Atom.__init__ waits for an instance of Var"
                              f", but it received '{v}'")


class Application(Term):  # the class of terms created with the second rule

    def __init__(self, t1, t2):
        if isinstance(t1, Term) and isinstance(t2, Term):
            self._data = (t1, t2)
        else:
            raise LambdaError("Application.__init__ waits for two instances"
                              f" of Term, but it received '{t1}', '{t2}'")


class Abstraction(Term):  # the class of terms created with the third rule

    def __init__(self, v, t):
        if isinstance(v, Var) and isinstance(t, Term):
            self._data = (v, t)
        else:
            raise LambdaError("Abstraction.__init__ waits for an instance of"
                              " Var and an instance of Term"
                              f", but it receive '{v}' and '{t}'")



from abc import ABC, abstractmethod
import random


class OneStepStrategy(ABC):
    @abstractmethod
    def redex_index(self, term: Term, init_index=0) -> int:
        """
        :return: index of the vertex of a subterm that has an outer redex.
                The index of a vertex is the index of this vertex in the topological sort of the tree vertices.
                Indexing starts at 1.
        """


class LOStrategy(OneStepStrategy):
    def redex_index(self, term: Term, init_index=0) -> int:
        if (term.kind == "atom") or (len(term.redexes) == 0):
            raise ValueError("The term doesn't contain a redex")
        if term.kind == "application":
            if term.is_beta_redex:
                return init_index + 1
            if len(term._data[0].redexes) != 0:
                return self.redex_index(term._data[0], init_index + 1)
            return self.redex_index(term._data[1],
                                    init_index + term._data[0].vertices_number + 1)
        # self is Abstraction:
        return self.redex_index(term._data[1], init_index + 1)


class DeanRandStrategy(OneStepStrategy):
    def redex_index(self, term: Term, init_index=0) -> int:
        count_redexes = len(term.redexes)
        print(f"count_redexes: {count_redexes}")
        if term.kind == "atom" or count_redexes == 0:
            raise ValueError("The term doesn't contain a redex")
        elif term.kind == "application":
            index = random.randint(0, count_redexes - 1)
            if term.is_beta_redex and index == 0:
                return init_index + 1
            elif len(term._data[0].redexes) >= index and len(term._data[0].redexes) != 0:
                return self.redex_index(term._data[0], init_index + 1)
            else:
                return self.redex_index(term._data[1], init_index + term._data[0].vertices_number + 1)
        else:
            return self.redex_index(term._data[1], init_index + 1)



Lambda = Abstraction
App = Application

# <span style="color: #A1F99C">TEST PROCEDURE</span>

In [2]:
rs = DeanRandStrategy()


def test_procedure(term, reduction_strategy=rs, print_steps=True,
                   steps_lim=60, assert_result=""):
    term = term._update_bound_vars()
    n_steps = 0
    is_term_normalized = True
    if print_steps:
        print(f"{n_steps}: {term}")
    while term.redexes:
        term = term._beta_conversion(reduction_strategy)
        term = term._update_bound_vars()
        n_steps += 1
        if print_steps:
            print(f"{n_steps}: {term}")
        if n_steps > steps_lim:
            is_term_normalized = False
            break

    print(f"\n\nreduction steps: {n_steps}")
    print(f"norm term:       {term}" if is_term_normalized else f"NOT norm term: {term}")

# <span style="color: #A1F99C">USEFUL TERMS</span>

In [3]:
def ite_term():
    x, y, c = Var(), Var(), Var()
    x_, y_, c_ = Atom(x), Atom(y), Atom(c)
    return Lambda(c, Lambda(x, Lambda(y, multi_app_term(c_, x_, y_))))


def true_term():
    x, y = Var(), Var()
    x_ = Atom(x)
    return Lambda(x, Lambda(y, x_))


def num_zero_term():
    s, z = Var(), Var()
    z_ = Atom(z)
    return Lambda(s, Lambda(z, z_))


def num_term(n: int):
    if n <= 0:
        return num_zero_term()
    s, z = Var(), Var()
    s_, z_ = Atom(s), Atom(z)
    core_term = App(s_, z_)
    for _ in range(n - 1):
        core_term = App(s_, core_term)
    return Lambda(s, Lambda(z, core_term))


def multi_app_term(term_0: Term, term_1: Term, *terms: Term):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term


def false_term():
    x, y = Var(), Var()
    y_ = Atom(y)
    return Lambda(x, Lambda(y, y_))


def pair_term():
    x, y, p = Var(), Var(), Var()
    x_, y_, p_ = Atom(x), Atom(y), Atom(p)
    return Lambda(x, Lambda(y, Lambda(p, multi_app_term(p_, x_, y_))))


def first_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, true_term()))


def second_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, false_term()))


def succ_term():
    x, y, n = Var(), Var(), Var()
    x_, y_, n_ = Atom(x), Atom(y), Atom(n)
    return Lambda(n, Lambda(x, Lambda(y, App(x_, multi_app_term(n_, x_, y_)))))


def sinc_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(
        p,
        multi_app_term(
            pair_term(),
            App(second_term(), p_),
            App(succ_term(), App(second_term(), p_)),
        ),
    )


def pred_term():
    n, f, x, g, h, u = Var(), Var(), Var(), Var(), Var(), Var()
    n_, f_, x_ = Atom(n), Atom(f), Atom(x)
    g_, h_, u_ = Atom(g), Atom(h), Atom(u)

    return Lambda(n, Lambda(f, Lambda(x, multi_app_term(
        n_,
        Lambda(g, Lambda(h, App(h_, App(g_, f_)))),
        Lambda(u, x_),
        Lambda(u, u_)
    ))))


def plus_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(m, Lambda(n, multi_app_term(n_, succ_term(), m_)))


def subtract_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(n, Lambda(m, multi_app_term(m_, pred_term(), n_)))


def iszero_term():
    x, n = Var(), Var()
    n_ = Atom(n)
    return Lambda(n, App(App(n_, Lambda(x, false_term())), true_term()))


def leq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n, Lambda(m, App(iszero_term(), multi_app_term(subtract_term(), n_, m_)))
    )


def and_term():
    a, b = Var(), Var()
    a_, b_ = Atom(a), Atom(b)
    return Lambda(a, Lambda(b, multi_app_term(ite_term(), a_, b_, a_)))


def eq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n,
        Lambda(
            m,
            multi_app_term(
                and_term(),
                multi_app_term(leq_term(), n_, m_),
                multi_app_term(leq_term(), m_, n_),
            ),
        ),
    )

# <span style="color: #A1F99C">1.</span>
<span style="color:#bb91f3">(λx. (λy.y)) ((λx. (xx)) (λx. (xx)))</span> == <span style="color:#f1bd47">I</span> == <span style="color:#f1bd47">λx. x</span>
<span style="color:#c9fafa">This term reduce in one step by LO, RO strategy
And reduce in the same terms with other strategy.</span>

In [4]:
def term_test_1():
    x, y = Var(), Var()
    x_, y_ = Atom(x), Atom(y)

    return App(
             Lambda(x, Lambda(y, y_)),
             App(
                 Lambda(x, App(x_, x_)),
                 Lambda(x, App(x_, x_))
             )
         )


test_procedure(term_test_1())

0: ((λ#2. (λ#3. #3)) ((λ#4. (#4 #4)) (λ#5. (#5 #5))))
count_redexes: 2
count_redexes: 1
1: ((λ#6. (λ#7. #7)) ((λ#8. (#8 #8)) (λ#9. (#9 #9))))
count_redexes: 2
count_redexes: 1
2: ((λ#10. (λ#11. #11)) ((λ#12. (#12 #12)) (λ#13. (#13 #13))))
count_redexes: 2
count_redexes: 1
3: ((λ#14. (λ#15. #15)) ((λ#16. (#16 #16)) (λ#17. (#17 #17))))
count_redexes: 2
4: (λ#18. #18)


reduction steps: 4
norm term:       (λ#18. #18)


# <span style="color: #A1F99C">2.</span>
<span style="color:#bb91f3">(ITE TRUE 3 1)</span> == <span style="color:#f1bd47>">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [5]:
def term_test_2():
    return multi_app_term(ite_term(), true_term(), num_term(3), num_term(1))


test_procedure(term_test_2())

0: ((((λ#28. (λ#29. (λ#30. ((#28 #29) #30)))) (λ#31. (λ#32. #31))) (λ#33. (λ#34. (#33 (#33 (#33 #34)))))) (λ#35. (λ#36. (#35 #36))))
count_redexes: 1
count_redexes: 1
count_redexes: 1
1: (((λ#37. (λ#38. (((λ#39. (λ#40. #39)) #37) #38))) (λ#41. (λ#42. (#41 (#41 (#41 #42)))))) (λ#43. (λ#44. (#43 #44))))
count_redexes: 2
count_redexes: 2
count_redexes: 1
count_redexes: 1
count_redexes: 1
count_redexes: 1
2: (((λ#45. (λ#46. ((λ#47. #45) #46))) (λ#48. (λ#49. (#48 (#48 (#48 #49)))))) (λ#50. (λ#51. (#50 #51))))
count_redexes: 2
count_redexes: 2
3: ((λ#52. ((λ#53. (λ#54. (λ#55. (#54 (#54 (#54 #55)))))) #52)) (λ#56. (λ#57. (#56 #57))))
count_redexes: 2
count_redexes: 1
count_redexes: 1
4: ((λ#58. (λ#59. (λ#60. (#59 (#59 (#59 #60)))))) (λ#61. (λ#62. (#61 #62))))
count_redexes: 1
5: (λ#63. (λ#64. (#63 (#63 (#63 #64)))))


reduction steps: 5
norm term:       (λ#63. (λ#64. (#63 (#63 (#63 #64)))))


# <span style="color: #A1F99C">3.</span>
<span style="color:#bb91f3">(ITE FALSE 0 4)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [6]:
def term_test_3():
    return multi_app_term(ite_term(), false_term(), num_term(0), num_term(4))


test_procedure(term_test_3())

0: ((((λ#74. (λ#75. (λ#76. ((#74 #75) #76)))) (λ#77. (λ#78. #78))) (λ#79. (λ#80. #80))) (λ#81. (λ#82. (#81 (#81 (#81 (#81 #82)))))))
count_redexes: 1
count_redexes: 1
count_redexes: 1
1: (((λ#83. (λ#84. (((λ#85. (λ#86. #86)) #83) #84))) (λ#87. (λ#88. #88))) (λ#89. (λ#90. (#89 (#89 (#89 (#89 #90)))))))
count_redexes: 2
count_redexes: 2
count_redexes: 1
count_redexes: 1
count_redexes: 1
count_redexes: 1
2: (((λ#91. (λ#92. ((λ#93. #93) #92))) (λ#94. (λ#95. #95))) (λ#96. (λ#97. (#96 (#96 (#96 (#96 #97)))))))
count_redexes: 2
count_redexes: 2
3: ((λ#98. ((λ#99. #99) #98)) (λ#100. (λ#101. (#100 (#100 (#100 (#100 #101)))))))
count_redexes: 2
count_redexes: 1
count_redexes: 1
4: ((λ#102. #102) (λ#103. (λ#104. (#103 (#103 (#103 (#103 #104)))))))
count_redexes: 1
5: (λ#105. (λ#106. (#105 (#105 (#105 (#105 #106))))))


reduction steps: 5
norm term:       (λ#105. (λ#106. (#105 (#105 (#105 (#105 #106))))))


# <span style="color: #A1F99C">4.</span>
<span style="color:#bb91f3">(PRED 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [7]:
def term_test_4():
    return multi_app_term(pred_term(), num_term(0))


test_procedure(term_test_4())

0: ((λ#115. (λ#116. (λ#117. (((#115 (λ#118. (λ#119. (#119 (#118 #116))))) (λ#120. #117)) (λ#121. #121))))) (λ#122. (λ#123. #123)))
count_redexes: 1
1: (λ#124. (λ#125. ((((λ#126. (λ#127. #127)) (λ#128. (λ#129. (#129 (#128 #124))))) (λ#130. #125)) (λ#131. #131))))
count_redexes: 1
count_redexes: 1
count_redexes: 1
count_redexes: 1
count_redexes: 1
2: (λ#132. (λ#133. (((λ#134. #134) (λ#135. #133)) (λ#136. #136))))
count_redexes: 1
count_redexes: 1
count_redexes: 1
count_redexes: 1
3: (λ#137. (λ#138. ((λ#139. #138) (λ#140. #140))))
count_redexes: 1
count_redexes: 1
count_redexes: 1
4: (λ#141. (λ#142. #142))


reduction steps: 4
norm term:       (λ#141. (λ#142. #142))


# <span style="color: #A1F99C">5.</span>
<span style="color:#bb91f3">(PRED 5)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [8]:
def term_test_5():
    return multi_app_term(pred_term(), num_term(5))


test_procedure(term_test_5())

0: ((λ#151. (λ#152. (λ#153. (((#151 (λ#154. (λ#155. (#155 (#154 #152))))) (λ#156. #153)) (λ#157. #157))))) (λ#158. (λ#159. (#158 (#158 (#158 (#158 (#158 #159))))))))
count_redexes: 1
1: (λ#160. (λ#161. ((((λ#162. (λ#163. (#162 (#162 (#162 (#162 (#162 #163))))))) (λ#164. (λ#165. (#165 (#164 #160))))) (λ#166. #161)) (λ#167. #167))))
count_redexes: 1
count_redexes: 1
count_redexes: 1
count_redexes: 1
count_redexes: 1
2: (λ#168. (λ#169. (((λ#170. ((λ#171. (λ#172. (#172 (#171 #168)))) ((λ#173. (λ#174. (#174 (#173 #168)))) ((λ#175. (λ#176. (#176 (#175 #168)))) ((λ#177. (λ#178. (#178 (#177 #168)))) ((λ#179. (λ#180. (#180 (#179 #168)))) #170)))))) (λ#181. #169)) (λ#182. #182))))
count_redexes: 6
count_redexes: 6
count_redexes: 6
count_redexes: 6
count_redexes: 5
count_redexes: 5
3: (λ#183. (λ#184. (((λ#185. (λ#186. (#186 (((λ#187. (λ#188. (#188 (#187 #183)))) ((λ#189. (λ#190. (#190 (#189 #183)))) ((λ#191. (λ#192. (#192 (#191 #183)))) ((λ#193. (λ#194. (#194 (#193 #183)))) #185)))) #183)))) (λ#1

# <span style="color: #A1F99C">6.</span>
<span style="color:#bb91f3">(PLUS 0 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [9]:
def term_test_6():
    return multi_app_term(plus_term(), num_term(0), num_term(0))


test_procedure(term_test_6())

0: (((λ#296. (λ#297. ((#297 (λ#298. (λ#299. (λ#300. (#299 ((#298 #299) #300)))))) #296))) (λ#301. (λ#302. #302))) (λ#303. (λ#304. #304)))
count_redexes: 1
count_redexes: 1
1: ((λ#305. ((#305 (λ#306. (λ#307. (λ#308. (#307 ((#306 #307) #308)))))) (λ#309. (λ#310. #310)))) (λ#311. (λ#312. #312)))
count_redexes: 1
2: (((λ#313. (λ#314. #314)) (λ#315. (λ#316. (λ#317. (#316 ((#315 #316) #317)))))) (λ#318. (λ#319. #319)))
count_redexes: 1
count_redexes: 1
3: ((λ#320. #320) (λ#321. (λ#322. #322)))
count_redexes: 1
4: (λ#323. (λ#324. #324))


reduction steps: 4
norm term:       (λ#323. (λ#324. #324))


# <span style="color: #A1F99C">7.</span>
<span style="color:#bb91f3">(PLUS 2 3)</span> == <span style="color:#f1bd47">5</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x (x y)))))</span>

In [10]:
def term_test_7():
    return multi_app_term(plus_term(), num_term(2), num_term(3))


test_procedure(term_test_7())

0: (((λ#334. (λ#335. ((#335 (λ#336. (λ#337. (λ#338. (#337 ((#336 #337) #338)))))) #334))) (λ#339. (λ#340. (#339 (#339 #340))))) (λ#341. (λ#342. (#341 (#341 (#341 #342))))))
count_redexes: 1
count_redexes: 1
1: ((λ#343. ((#343 (λ#344. (λ#345. (λ#346. (#345 ((#344 #345) #346)))))) (λ#347. (λ#348. (#347 (#347 #348)))))) (λ#349. (λ#350. (#349 (#349 (#349 #350))))))
count_redexes: 1
2: (((λ#351. (λ#352. (#351 (#351 (#351 #352))))) (λ#353. (λ#354. (λ#355. (#354 ((#353 #354) #355)))))) (λ#356. (λ#357. (#356 (#356 #357)))))
count_redexes: 1
count_redexes: 1
3: ((λ#358. ((λ#359. (λ#360. (λ#361. (#360 ((#359 #360) #361))))) ((λ#362. (λ#363. (λ#364. (#363 ((#362 #363) #364))))) ((λ#365. (λ#366. (λ#367. (#366 ((#365 #366) #367))))) #358)))) (λ#368. (λ#369. (#368 (#368 #369)))))
count_redexes: 4
count_redexes: 3
count_redexes: 3
4: ((λ#370. (λ#371. (λ#372. (#371 ((((λ#373. (λ#374. (λ#375. (#374 ((#373 #374) #375))))) ((λ#376. (λ#377. (λ#378. (#377 ((#376 #377) #378))))) #370)) #371) #372))))) (λ#37

# <span style="color: #A1F99C">8.</span>
<span style="color:#bb91f3">(SUBTRACT 4 1)</span> == <span style="color:#f1bd47">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [11]:
def term_test_8():
    return multi_app_term(subtract_term(), num_term(4), num_term(1))


test_procedure(term_test_8())

0: (((λ#447. (λ#448. ((#448 (λ#449. (λ#450. (λ#451. (((#449 (λ#452. (λ#453. (#453 (#452 #450))))) (λ#454. #451)) (λ#455. #455)))))) #447))) (λ#456. (λ#457. (#456 (#456 (#456 (#456 #457))))))) (λ#458. (λ#459. (#458 #459))))
count_redexes: 1
count_redexes: 1
1: ((λ#460. ((#460 (λ#461. (λ#462. (λ#463. (((#461 (λ#464. (λ#465. (#465 (#464 #462))))) (λ#466. #463)) (λ#467. #467)))))) (λ#468. (λ#469. (#468 (#468 (#468 (#468 #469)))))))) (λ#470. (λ#471. (#470 #471))))
count_redexes: 1
2: (((λ#472. (λ#473. (#472 #473))) (λ#474. (λ#475. (λ#476. (((#474 (λ#477. (λ#478. (#478 (#477 #475))))) (λ#479. #476)) (λ#480. #480)))))) (λ#481. (λ#482. (#481 (#481 (#481 (#481 #482)))))))
count_redexes: 1
count_redexes: 1
3: ((λ#483. ((λ#484. (λ#485. (λ#486. (((#484 (λ#487. (λ#488. (#488 (#487 #485))))) (λ#489. #486)) (λ#490. #490))))) #483)) (λ#491. (λ#492. (#491 (#491 (#491 (#491 #492)))))))
count_redexes: 2
count_redexes: 1
count_redexes: 1
4: ((λ#493. (λ#494. (λ#495. (((#493 (λ#496. (λ#497. (#497 (#496 #494

# <span style="color: #A1F99C">9.</span>
<span style="color:#bb91f3">(SUBTRACT 3 5)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [12]:
def term_test_9():
    return multi_app_term(subtract_term(), num_term(3), num_term(5))


test_procedure(term_test_9())

0: (((λ#612. (λ#613. ((#613 (λ#614. (λ#615. (λ#616. (((#614 (λ#617. (λ#618. (#618 (#617 #615))))) (λ#619. #616)) (λ#620. #620)))))) #612))) (λ#621. (λ#622. (#621 (#621 (#621 #622)))))) (λ#623. (λ#624. (#623 (#623 (#623 (#623 (#623 #624))))))))
count_redexes: 1
count_redexes: 1
1: ((λ#625. ((#625 (λ#626. (λ#627. (λ#628. (((#626 (λ#629. (λ#630. (#630 (#629 #627))))) (λ#631. #628)) (λ#632. #632)))))) (λ#633. (λ#634. (#633 (#633 (#633 #634))))))) (λ#635. (λ#636. (#635 (#635 (#635 (#635 (#635 #636))))))))
count_redexes: 1
2: (((λ#637. (λ#638. (#637 (#637 (#637 (#637 (#637 #638))))))) (λ#639. (λ#640. (λ#641. (((#639 (λ#642. (λ#643. (#643 (#642 #640))))) (λ#644. #641)) (λ#645. #645)))))) (λ#646. (λ#647. (#646 (#646 (#646 #647))))))
count_redexes: 1
count_redexes: 1
3: ((λ#648. ((λ#649. (λ#650. (λ#651. (((#649 (λ#652. (λ#653. (#653 (#652 #650))))) (λ#654. #651)) (λ#655. #655))))) ((λ#656. (λ#657. (λ#658. (((#656 (λ#659. (λ#660. (#660 (#659 #657))))) (λ#661. #658)) (λ#662. #662))))) ((λ#663. (λ

# <span style="color: #A1F99C">10.</span>
<span style="color:#bb91f3">(LEQ 3 2)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [13]:
def term_test_10():
    return multi_app_term(leq_term(), num_term(3), num_term(2))


test_procedure(term_test_10())

0: (((λ#1627. (λ#1628. ((λ#1629. ((#1629 (λ#1630. (λ#1631. (λ#1632. #1632)))) (λ#1633. (λ#1634. #1633)))) (((λ#1635. (λ#1636. ((#1636 (λ#1637. (λ#1638. (λ#1639. (((#1637 (λ#1640. (λ#1641. (#1641 (#1640 #1638))))) (λ#1642. #1639)) (λ#1643. #1643)))))) #1635))) #1627) #1628)))) (λ#1644. (λ#1645. (#1644 (#1644 (#1644 #1645)))))) (λ#1646. (λ#1647. (#1646 (#1646 #1647)))))
count_redexes: 3
count_redexes: 3
1: ((λ#1648. ((λ#1649. ((#1649 (λ#1650. (λ#1651. (λ#1652. #1652)))) (λ#1653. (λ#1654. #1653)))) (((λ#1655. (λ#1656. ((#1656 (λ#1657. (λ#1658. (λ#1659. (((#1657 (λ#1660. (λ#1661. (#1661 (#1660 #1658))))) (λ#1662. #1659)) (λ#1663. #1663)))))) #1655))) (λ#1664. (λ#1665. (#1664 (#1664 (#1664 #1665)))))) #1648))) (λ#1666. (λ#1667. (#1666 (#1666 #1667)))))
count_redexes: 3
2: ((λ#1668. ((#1668 (λ#1669. (λ#1670. (λ#1671. #1671)))) (λ#1672. (λ#1673. #1672)))) (((λ#1674. (λ#1675. ((#1675 (λ#1676. (λ#1677. (λ#1678. (((#1676 (λ#1679. (λ#1680. (#1680 (#1679 #1677))))) (λ#1681. #1678)) (λ#1682. #1682)

# <span style="color: #A1F99C">11.</span>
<span style="color:#bb91f3">(LEQ 2 5)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [14]:
def term_test_11():
    return multi_app_term(leq_term(), num_term(2), num_term(5))


test_procedure(term_test_11())

0: (((λ#2161. (λ#2162. ((λ#2163. ((#2163 (λ#2164. (λ#2165. (λ#2166. #2166)))) (λ#2167. (λ#2168. #2167)))) (((λ#2169. (λ#2170. ((#2170 (λ#2171. (λ#2172. (λ#2173. (((#2171 (λ#2174. (λ#2175. (#2175 (#2174 #2172))))) (λ#2176. #2173)) (λ#2177. #2177)))))) #2169))) #2161) #2162)))) (λ#2178. (λ#2179. (#2178 (#2178 #2179))))) (λ#2180. (λ#2181. (#2180 (#2180 (#2180 (#2180 (#2180 #2181))))))))
count_redexes: 3
count_redexes: 3
1: ((λ#2182. ((λ#2183. ((#2183 (λ#2184. (λ#2185. (λ#2186. #2186)))) (λ#2187. (λ#2188. #2187)))) (((λ#2189. (λ#2190. ((#2190 (λ#2191. (λ#2192. (λ#2193. (((#2191 (λ#2194. (λ#2195. (#2195 (#2194 #2192))))) (λ#2196. #2193)) (λ#2197. #2197)))))) #2189))) (λ#2198. (λ#2199. (#2198 (#2198 #2199))))) #2182))) (λ#2200. (λ#2201. (#2200 (#2200 (#2200 (#2200 (#2200 #2201))))))))
count_redexes: 3
2: ((λ#2202. ((#2202 (λ#2203. (λ#2204. (λ#2205. #2205)))) (λ#2206. (λ#2207. #2206)))) (((λ#2208. (λ#2209. ((#2209 (λ#2210. (λ#2211. (λ#2212. (((#2210 (λ#2213. (λ#2214. (#2214 (#2213 #2211))))) 

# <span style="color: #A1F99C">12.</span>
<span style="color:#bb91f3">(EQ 2 2)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [17]:
def term_test_12():
    return multi_app_term(eq_term(), num_term(2), num_term(2))


test_procedure(term_test_12(), steps_lim=100)

0: (((λ#9426. (λ#9427. (((λ#9428. (λ#9429. ((((λ#9430. (λ#9431. (λ#9432. ((#9430 #9431) #9432)))) #9428) #9429) #9428))) (((λ#9433. (λ#9434. ((λ#9435. ((#9435 (λ#9436. (λ#9437. (λ#9438. #9438)))) (λ#9439. (λ#9440. #9439)))) (((λ#9441. (λ#9442. ((#9442 (λ#9443. (λ#9444. (λ#9445. (((#9443 (λ#9446. (λ#9447. (#9447 (#9446 #9444))))) (λ#9448. #9445)) (λ#9449. #9449)))))) #9441))) #9433) #9434)))) #9426) #9427)) (((λ#9450. (λ#9451. ((λ#9452. ((#9452 (λ#9453. (λ#9454. (λ#9455. #9455)))) (λ#9456. (λ#9457. #9456)))) (((λ#9458. (λ#9459. ((#9459 (λ#9460. (λ#9461. (λ#9462. (((#9460 (λ#9463. (λ#9464. (#9464 (#9463 #9461))))) (λ#9465. #9462)) (λ#9466. #9466)))))) #9458))) #9450) #9451)))) #9427) #9426)))) (λ#9467. (λ#9468. (#9467 (#9467 #9468))))) (λ#9469. (λ#9470. (#9469 (#9469 #9470)))))
count_redexes: 9
count_redexes: 9
count_redexes: 8
count_redexes: 8
count_redexes: 8
count_redexes: 5
1: (((λ#9471. (λ#9472. ((λ#9473. ((((λ#9474. (λ#9475. (λ#9476. ((#9474 #9475) #9476)))) (((λ#9477. (λ#9478. ((λ

# <span style="color: #A1F99C">13.</span>
<span style="color:#bb91f3">(EQ 1 4)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [18]:
def term_test_13():
    return multi_app_term(eq_term(), num_term(1), num_term(4))


test_procedure(term_test_13(), steps_lim=100)

0: (((λ#12453. (λ#12454. (((λ#12455. (λ#12456. ((((λ#12457. (λ#12458. (λ#12459. ((#12457 #12458) #12459)))) #12455) #12456) #12455))) (((λ#12460. (λ#12461. ((λ#12462. ((#12462 (λ#12463. (λ#12464. (λ#12465. #12465)))) (λ#12466. (λ#12467. #12466)))) (((λ#12468. (λ#12469. ((#12469 (λ#12470. (λ#12471. (λ#12472. (((#12470 (λ#12473. (λ#12474. (#12474 (#12473 #12471))))) (λ#12475. #12472)) (λ#12476. #12476)))))) #12468))) #12460) #12461)))) #12453) #12454)) (((λ#12477. (λ#12478. ((λ#12479. ((#12479 (λ#12480. (λ#12481. (λ#12482. #12482)))) (λ#12483. (λ#12484. #12483)))) (((λ#12485. (λ#12486. ((#12486 (λ#12487. (λ#12488. (λ#12489. (((#12487 (λ#12490. (λ#12491. (#12491 (#12490 #12488))))) (λ#12492. #12489)) (λ#12493. #12493)))))) #12485))) #12477) #12478)))) #12454) #12453)))) (λ#12494. (λ#12495. (#12494 #12495)))) (λ#12496. (λ#12497. (#12496 (#12496 (#12496 (#12496 #12497)))))))
count_redexes: 9
count_redexes: 9
count_redexes: 8
count_redexes: 8
count_redexes: 8
count_redexes: 5
1: (((λ#12498. 